# Obtain and Append Data

This notebook contains code to obtain and append financial data to avoid going over API request limits.

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import plotly.graph_objects as go

import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import numpy as np
from IPython.display import clear_output
import time
# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [2]:
import platform
print(platform.python_version())

3.8.3


In [3]:
# Obtain the API credentials from an external hidden file

from passwords import api_key

In [4]:
# A function to obtain price history

#Utilizing "UniBit API" by Vincent Jiang
#Site: https://unibit.ai/product

#Errors may occur if credits run out, add an error code check to this function

def retrieve_ticker_price_info(ticker, start_date, end_date = False):
    
    # Obtain today's date
    today = date.today()
    
    # A condition to set the end date to today if there is no end date given
    if not end_date:
        end_date = today
    
    
    url = 'https://api.unibit.ai/v2/stock/historical/?tickers={}&interval=1&startDate={}&endDate={}&selectedFields=all&dataType=json&accessKey={}'.format(ticker, start_date, end_date, api_key)
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print("The response status code is: " + str(response.status_code))
        return None
    
    return response.json()
    

In [5]:
def retrieve_price_history_dataframe(ticker, ticker_price_history):
    
    data_frame = pd.DataFrame(ticker_price_history['result_data'][ticker])
    
    # Set the index to the 'date' column
    data_frame.set_index('date', inplace = True)
    
    data_frame.sort_index(inplace=True)
    
    return data_frame

In [6]:
today = date.today()
a_year_ago = str(date.today() - timedelta(days=1*365))
some_five_years_ago = str(date.today() - timedelta(days=5*365))

## Obtain Original Data from the API

**NOTE: Skip this section if you already have a data set**

In [7]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
inx_info = retrieve_ticker_price_info('^GSPC', some_five_years_ago)

# Create and view the dataframe
inx_df = retrieve_price_history_dataframe('^GSPC', inx_info)
inx_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2015-08-31,3915100000,1986.73,1965.98,1972.18,1972.18,1986.73
2015-09-01,4371850000,1970.09,1903.07,1913.85,1913.85,1970.09
2015-09-02,3742620000,1948.91,1916.52,1948.86,1948.86,1916.52
2015-09-03,3520700000,1975.01,1944.72,1951.13,1951.13,1950.79
2015-09-04,3167090000,1947.76,1911.21,1921.22,1921.22,1947.76


In [8]:
# Save the data as a csv
# Keep the index as it contains the dates
#inx_df.to_csv('data_files/financial_data/GSPC.csv')
# five years
inx_df.to_csv('data_files/financial_data/august_31_2015_to_now_GSPC.csv')

## Import a CSV of Time Series Data

This code assumes the data is in ascending order, if this is not the case then make the relevant adjustments. 

In [7]:
inx_df = pd.read_csv('data_files/financial_data/august_31_2015_to_now_GSPC.csv', index_col = 'date')

# Convert index to date
# Comment this out if the index is already date type
inx_df.index = pd.to_datetime(inx_df.index).date

inx_df.head()

,volume,high,low,adj_close,close,open
2015-08-31,3915100000,1986.73,1965.98,1972.18,1972.18,1986.73
2015-09-01,4371850000,1970.09,1903.07,1913.85,1913.85,1970.09
2015-09-02,3742620000,1948.91,1916.52,1948.86,1948.86,1916.52
2015-09-03,3520700000,1975.01,1944.72,1951.13,1951.13,1950.79
2015-09-04,3167090000,1947.76,1911.21,1921.22,1921.22,1947.76


In [8]:
# Obtain the most recent date
inx_df.index.max()

datetime.date(2020, 10, 9)

## Obtain Most Recent Data From The API

In [9]:
# Obtain the next day after the series ends, as a string
next_day_series_ends = str(inx_df.index.max() + timedelta(days=1))
next_day_series_ends

'2020-10-10'

In [10]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
additional_info = retrieve_ticker_price_info('^GSPC', next_day_series_ends)

# Create and view the dataframe
additional_info_df = retrieve_price_history_dataframe('^GSPC', additional_info)
additional_info_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2020-10-12,2243800302,3549.85,3499.61,3534.22,3534.22,3500.02
2020-10-13,2357875709,3534.01,3500.86,3511.93,3511.93,3534.01
2020-10-14,2183733760,3527.94,3480.55,3488.67,3488.67,3515.47
2020-10-15,2009665085,3489.08,3440.89,3483.34,3483.34,3453.72
2020-10-16,2221194565,3515.76,3480.45,3483.81,3483.81,3493.50


In [11]:
additional_info_df.tail()

,volume,high,low,adj_close,close,open
date,,,,,,
2020-12-14,2508893728,3697.61,3645.84,3647.49,3647.49,3675.27
2020-12-15,2505148998,3695.29,3659.62,3694.62,3694.62,3666.41
2020-12-16,2218203483,3711.27,3688.57,3701.17,3701.17,3696.25
2020-12-17,1632783836,3725.12,3710.87,3720.74,3720.74,3713.65
2020-12-18,4190993787,3726.70,3685.84,3709.41,3709.41,3722.39


## Append The Original Dataframe

You can either add to the old file or create a new one

In [12]:
result = pd.concat([inx_df, additional_info_df])

In [13]:
result.tail(15)

,volume,high,low,adj_close,close,open
2020-11-30,3597012832,3634.18,3594.39,3621.63,3621.63,3634.18
2020-12-01,2586912631,3678.45,3645.87,3662.45,3662.45,3645.87
2020-12-02,2355582608,3670.96,3644.84,3669.01,3669.01,3653.78
2020-12-03,2616729365,3682.73,3657.17,3666.72,3666.72,3668.28
2020-12-04,2556167753,3699.20,3670.94,3699.12,3699.12,3670.94
2020-12-07,2320415964,3697.41,3678.88,3691.96,3691.96,3694.73
2020-12-08,2247762425,3708.45,3678.83,3702.25,3702.25,3683.05
2020-12-09,2726403479,3712.39,3660.54,3672.82,3672.82,3705.98
2020-12-10,2380891125,3678.49,3645.18,3668.10,3668.10,3659.13
2020-12-11,2292104467,3665.91,3633.40,3663.46,3663.46,3656.08


## Check The Dataframe for Duplicates

In [14]:
result[result.index.duplicated()]

,volume,high,low,adj_close,close,open


## Save The New Dataframe

You can overwrite the old file as well

In [15]:
# Save the data as a csv
result.to_csv('data_files/financial_data/GSPC_newdata.csv', index_label = 'date')

In [16]:
# Read in the file
pd.read_csv('data_files/financial_data/GSPC_newdata.csv', index_col = 'date').tail(15)

,volume,high,low,adj_close,close,open
date,,,,,,
2020-11-30,3597012832,3634.18,3594.39,3621.63,3621.63,3634.18
2020-12-01,2586912631,3678.45,3645.87,3662.45,3662.45,3645.87
2020-12-02,2355582608,3670.96,3644.84,3669.01,3669.01,3653.78
2020-12-03,2616729365,3682.73,3657.17,3666.72,3666.72,3668.28
2020-12-04,2556167753,3699.20,3670.94,3699.12,3699.12,3670.94
2020-12-07,2320415964,3697.41,3678.88,3691.96,3691.96,3694.73
2020-12-08,2247762425,3708.45,3678.83,3702.25,3702.25,3683.05
2020-12-09,2726403479,3712.39,3660.54,3672.82,3672.82,3705.98
2020-12-10,2380891125,3678.49,3645.18,3668.10,3668.10,3659.13
